# Gradient-weighted Class Activation Mapping (Grad-CAM)

Given an input image, `Grad-CAM` is a technique to visualize **important** regions of the image. These regions are **important** because they can help a Convolutional Neural Netword(CNN)-based model to make the right prediction. That is, `Grad-CAM` can give us reasonable **visual explanations** of CNN-based models' predictions.





## Class Activation Mapping(CAM)

To understand `Grad-CAM`, we should firstly figure out how to generate `class activation maps` using `global average pooling (GAP)` in CNNs.

For a particular class $c$, a `class activation map` shows the **important** image regions used by CNN to identify the class $c$. For example:

![cam_1](./image/cam_1.png "cam_1")

To generate a class activation map, we only need to take a pre-trained CNN and do the classification. Given a input image, when going through several convolutional layers, the image shrinks spatially while the features increase.  After the feature extraction, we have many small feature maps. 

![cam_2](./image/cam_2.png "cam_2")

Suppose we have $n$ feature maps of spatial size $7\times 7$ before the final **global** average pooling. After `GAP`, we have $n$ feature maps of spatial size $1\times 1$. If we have $1000$ classes in total, the weights $W$ would be of shape $(n, 1000)$. 

To generate class activation map, we only consider one class at a time. For a particular class, it has $n$ weights $w_1, w_2, \dots, w_{n}$ for $n$ features. If a weight, say $w_{27}$, is greater than $0$, the corresponding feature $f_{27}$ is positively correlated with this class. If $w_{27} \leq 0$, $f_{27}$ makes the image not or less likely to belong to this class.

Then we can generate a generate class activition map:

Given:

1. a particular class and corresponding weights $w_1, w_2, \dots, w_{n}$

2. the feature maps of the last convolutional layer, i.e. $n$ feature maps of spatial size $7\times 7$, denoted as $A_1, A_2,\dots, A_{n}$

The result class activation map is a $7\times 7$ heat map, i.e., a weighted sum of feature maps:

$$w_1A_1 + w_2A_2 + \dots + w_{n}A_{n}$$

If the original input image is of size $224\times 224$, we need to rescale the $7\times 7$ map to $224\times 224$, and then plot the input image and the map over each other:

![cam_3](./image/cam_3.png "cam_3")

Above is the basic idea of `CAM`. And we can see that `CAM` is only applicable to particular CNN architectures, which use `GAP` to replace FC layer, i.e.,

$$feature\ maps\rightarrow global\ average\ pooling \rightarrow softmax\ layer$$

## Grad-CAM

`Grad-CAM` is a generalization of `CAM`, and it is applicable to a wide variety of CNN-based models. `Grad-CAM` can be used on many computer vision tasks such as image classification, image captioning and visual question answering. Here we only consider the case of image classification:

![grad_cam_1](./image/grad_cam_1.png "grad_cam_1")

After the last convolutional layer, we have $n$ feature maps $A_1, A_2, \dots A_n$, and each map $A_k$ is of spatial size $u\times v$. 

Similar to `CAM`, we only consider a particular class $c$ at a time, and the result of Grad-CAM is also a a weighted sum of feature maps, i.e.,

$$\alpha_1^cA_1 + \alpha_2^cA_2 + \dots + \alpha_{n}^cA_{n} = \sum_{k=1}^{n}\alpha_k^cA_k$$

Our task is to compute the weight $\alpha_k$.

Recap that to compute $w_k$($\alpha_k$), `CAM` uses `GAP` to replace FC layer , while for `Grad-CAM` we need to retain the FC layer. Without modifying the structure of CNN, we can compute $\alpha_k$ like this:

$$\alpha^c_k =\frac{1}{Z} \sum_{i = 1}^{u}\sum_{j = 1}^{v}\frac{{\partial{y^c}}}{{\partial A_{ij}^k}}$$

where $y^c$ is the score for class $c$, $Z = u\times v$ and the gradient $\frac{{\partial{y^c}}}{{\partial A_{ij}^k}}$ can be computed by propagation.

Then result $\sum_{k=1}^{n}\alpha_k^cA_k$ is a heatmap containing $u \times v$ pixels. And because we are only interested in the features that have a positive influence on the class of interest, we finally apply a ReLU to each pixel of the heapmap:

$$ReLU(\sum_{k=1}^{n}\alpha_k^cA_k)$$

Above is the basic idea of `Grad-CAM`.

## Reference 

[Paper on CAM]`https://arxiv.org/pdf/1512.04150.pdf`

[An Introduction Video of CAM]`https://www.youtube.com/watch?v=vTY58-51XZA`

[Paper on Grad-CAM]`https://arxiv.org/pdf/1610.02391.pdf`